# JAL-AM vs IQL

In [ ]:
import matplotlib.pyplot as plt
from multiprocessing.dummy import Pool as ThreadPool
from multiprocessing import cpu_count
import numpy as np
import pickle
import time

from games.foraging import Foraging
from agents.jalam_agent import JALAMAgent, JALAMAgentConfig
from agents.iql_agent import IQLAgent, IQLAgentConfig

from auxiliar.explorative_agents_training_and_eval import entrenar_agente_tarea, eval_experiments

## Definir configuración, entrenar y evaluar

In [ ]:
# Configuraciones de los juegos
game_config = [
        "Foraging-5x5-2p-1f-v3",
        "Foraging-5x5-3p-1f-coop-v3",
    ]

In [ ]:
configurations = {
    'fast_jalam_vs_iql_0': {
        'game': game_config[0],
        'agent_0': IQLAgentConfig(
            start_epsilon=1.0,          # Epsilon inicial
            min_epsilon=0.1,            # Mínimo valor que puede tomar Epsilon
            epsilon_decay = 0.9997,     # Decaimiento lineal
            alpha=0.15,                 # Taza de aprendizaje
            gamma=0.95,                 # Factor de descuento
            learn=True,                 # Variable que determina si se esta en étapa de exploración o no (update y Epsilon-greedy o política greedy)
            seed=1                      # Semilla para la aleatoriedad
        ),
        'agent_1': JALAMAgentConfig(
            start_epsilon=1.0,          # Epsilon inicial
            min_epsilon=0.1,            # Mínimo valor que puede tomar Epsilon
            epsilon_decay = 0.9997,     # Decaimiento lineal
            alpha=0.15,                 # Taza de aprendizaje
            gamma=0.95,                 # Factor de descuento
            learn=True,                 # Variable que determina si se esta en étapa de exploración o no (update y Epsilon-greedy o política greedy)
            seed=1                      # Semilla para la aleatoriedad
        ),
        'train_config': {
            'episodes': 1500,
            'iterations': 10,
        }
    },
    'fast_jalam_coop_1': {
        'game': game_config[0],
        'agent_0': IQLAgentConfig(
            start_epsilon=1.0,      # Epsilon inicial
            min_epsilon=0.1,        # Mínimo valor que puede tomar Epsilon
            epsilon_decay = 0.9999, # Decaimiento lineal
            alpha=0.2,              # Taza de aprendizaje
            gamma=0.95,             # Factor de descuento
            learn=True,             # Variable que determina si se esta en étapa de exploración o no (update y Epsilon-greedy o política greedy)
            seed=1                  # Semilla para la aleatoriedad
        ),
        'agent_1': JALAMAgentConfig(
            start_epsilon=1.0,      # Epsilon inicial
            min_epsilon=0.1,        # Mínimo valor que puede tomar Epsilon
            epsilon_decay = 0.9999, # Decaimiento lineal
            alpha=0.2,              # Taza de aprendizaje
            gamma=0.95,             # Factor de descuento
            learn=True,             # Variable que determina si se esta en étapa de exploración o no (update y Epsilon-greedy o política greedy)
            seed=1                  # Semilla para la aleatoriedad
        ),
        'agent_2': JALAMAgentConfig(
            start_epsilon=1.0,      # Epsilon inicial
            min_epsilon=0.1,        # Mínimo valor que puede tomar Epsilon
            epsilon_decay = 0.15, # Decaimiento lineal
            alpha=0.2,              # Taza de aprendizaje
            gamma=0.95,             # Factor de descuento
            learn=True,             # Variable que determina si se esta en étapa de exploración o no (update y Epsilon-greedy o política greedy)
            seed=1                  # Semilla para la aleatoriedad
        ),
        'train_config': {
            'episodes': 2000,
            'iterations': 15,
        }
    },
    'fast_jalam_coop_2': {
        'game': game_config[1],
        'agent_0': IQLAgentConfig(
            start_epsilon=1.0,      # Epsilon inicial
            min_epsilon=0.1,        # Mínimo valor que puede tomar Epsilon
            epsilon_decay = 0.9999, # Decaimiento lineal
            alpha=0.2,              # Taza de aprendizaje
            gamma=0.95,             # Factor de descuento
            learn=True,             # Variable que determina si se esta en étapa de exploración o no (update y Epsilon-greedy o política greedy)
            seed=1                  # Semilla para la aleatoriedad
        ),
        'agent_1': JALAMAgentConfig(
            start_epsilon=1.0,      # Epsilon inicial
            min_epsilon=0.1,        # Mínimo valor que puede tomar Epsilon
            epsilon_decay = 0.9999, # Decaimiento lineal
            alpha=0.2,              # Taza de aprendizaje
            gamma=0.95,             # Factor de descuento
            learn=True,             # Variable que determina si se esta en étapa de exploración o no (update y Epsilon-greedy o política greedy)
            seed=1                  # Semilla para la aleatoriedad
        ),
        'agent_2': JALAMAgentConfig(
            start_epsilon=1.0,      # Epsilon inicial
            min_epsilon=0.1,        # Mínimo valor que puede tomar Epsilon
            epsilon_decay = 0.15, # Decaimiento lineal
            alpha=0.2,              # Taza de aprendizaje
            gamma=0.95,             # Factor de descuento
            learn=True,             # Variable que determina si se esta en étapa de exploración o no (update y Epsilon-greedy o política greedy)
            seed=1                  # Semilla para la aleatoriedad
        ),
        'train_config': {
            'episodes': 2000,
            'iterations': 15,
        }
    }
}

In [ ]:
# Alias o nombre de los agentes para cada configuración
agent_alias = {'fast_jalam_vs_iql_0': {'agent_0': 'IQLAgent',     'agent_1': 'JALAMAgent'},
               'fast_jalam_coop_1':   {'agent_0': 'IQLAgent',     'agent_1': 'JALAMAgent_0', 'agent_2': 'JALAMAgent_1'},
               'fast_jalam_coop_2':   {'agent_0': 'JALAMAgent_0', 'agent_1': 'JALAMAgent_1', 'agent_2': 'JALAMAgent_2'}}

# Clases de los agentes definidas para cada configuración
agent_classes = {'fast_jalam_vs_iql_0': {'agent_0': IQLAgent,   'agent_1': JALAMAgent},
                 'fast_jalam_coop_1':   {'agent_0': IQLAgent,   'agent_1': JALAMAgent, 'agent_2': JALAMAgent},
                 'fast_jalam_coop_2':   {'agent_0': JALAMAgent, 'agent_1': JALAMAgent, 'agent_2': JALAMAgent}}

## Definir experimentos y entrenar

In [ ]:
# Armar lista de tareas
tareas = [(config_id, config, agent_classes[config_id], agent_alias[config_id]) for config_id, config in configurations.items() if config_id]
config_keys = [tarea[0] for tarea in tareas]

# Número de procesos
n_proc = min(len(tareas), cpu_count())

In [ ]:
with ThreadPool(n_proc) as pool:
    resultados = pool.map(entrenar_agente_tarea, tareas)

## Evaluar experimentos

In [ ]:
experiments_names = config_keys # Usar nombre de las tareas recién ejecutadas o elegir a mano
eval_experiments(experiments_names)